<a href="https://colab.research.google.com/github/gabrielcerono/GlioblastomaMultiforme/blob/main/Beredsen_survival.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk

In [33]:
!pip install scipy --upgrade

In [28]:
!pip install --upgrade pip
!pip uninstall --yes --quiet osqp
!pip install -U scikit-survival

  Using cached scikit-survival-0.15.0.post0.tar.gz (2.4 MB)
  distutils: /usr/lib/python3/dist-packages
  sysconfig: /usr/lib/python3.7/site-packages
  user = False
  home = None
  root = None
  prefix = None
  distutils: /usr/lib/python3/dist-packages
  sysconfig: /usr/lib/python3.7/site-packages
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  Using cached osqp-0.6.2.post0-cp37-cp37m-manylinux2014_x86_64.whl (212 kB)


In [ ]:
pip install eli5

In [3]:
from scipy.stats import rankdata


In [4]:
dataset = pd.read_excel('/content/Beredsen.xlsx')

In [5]:
dataset.columns

Index(['age', 'biopsy_debulking', 'KPS', 'adjuvant_treatment', 'SVZ status',
       'survival ', 'censoring', 'Unnamed: 7', 'Variables'],
      dtype='object')

In [6]:
dataset.drop(columns = ['Unnamed: 7', 'Variables'], inplace = True)

In [7]:
dataset.head()

,age,biopsy_debulking,KPS,adjuvant_treatment,SVZ status,survival,censoring
0,82,1.0,1.0,1.0,0.0,176.0,1.0
1,67,1.0,1.0,2.0,0.0,362.0,0.0
2,70,1.0,1.0,2.0,0.0,433.0,1.0
3,49,2.0,1.0,1.0,0.0,117.0,1.0
4,78,2.0,1.0,1.0,0.0,326.0,1.0


In [8]:
dataset = dataset.rename(columns={'censoring': 'death', 'survival ' : 'OS_time'})

In [9]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [10]:
imp_mean = IterativeImputer(random_state=0)

In [11]:
data = imp_mean.fit_transform(dataset)

In [12]:
data = pd.DataFrame(data=data, columns= dataset.columns,)

In [13]:
data.corr(method = 'pearson')

,age,biopsy_debulking,KPS,adjuvant_treatment,SVZ status,OS_time,death
age,1.000000,-0.082257,-0.161910,-0.029705,0.061247,-0.362111,0.173820
biopsy_debulking,-0.082257,1.000000,0.148640,-0.013053,-0.192748,0.363858,-0.144109
KPS,-0.161910,0.148640,1.000000,0.041573,-0.105957,0.297927,-0.127053
adjuvant_treatment,-0.029705,-0.013053,0.041573,1.000000,-0.015329,0.054873,-0.038990
SVZ status,0.061247,-0.192748,-0.105957,-0.015329,1.000000,-0.260603,0.110869
OS_time,-0.362111,0.363858,0.297927,0.054873,-0.260603,1.000000,-0.476101
death,0.173820,-0.144109,-0.127053,-0.038990,0.110869,-0.476101,1.000000


## Survival analysis and ranking

In [14]:
Y = data[['death', 'OS_time']]

In [15]:
Y['death'] = Y['death'].map({1.0: True, 0: False})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
Y = Y.to_records(index=False)

In [17]:
Y

rec.array([( True,  176.), (False,  362.), ( True,  433.), ( True,  117.),
           ( True,  326.), (False,  469.), ( True,  295.), (False, 1000.),
           (False,  680.), ( True,  474.), (False, 1000.), (False, 1000.),
           (False, 1000.), ( True,  130.), ( True,  411.), ( True,  230.),
           ( True,  277.), ( True,   83.), ( True,   23.), ( True,   61.),
           (False, 1000.), ( True,  161.), ( True,    9.), ( True,   76.),
           ( True,  508.), ( True,   50.), ( True,  377.), (False, 1000.),
           ( True,  230.), ( True,  320.), ( True,  122.), (False,   56.),
           (False, 1000.), ( True,   13.), (False,  211.), ( True,  253.),
           ( True,   74.), ( True,  303.), ( True,   97.), ( True,   15.),
           ( True,  241.), ( True,  173.), ( True,  843.), ( True,   54.),
           (False, 1000.), (False,  491.), ( True,  175.), (False,   11.),
           (False,  611.), ( True,  340.), (False,  490.), (False,  275.),
           (False,  345.)

In [18]:
X = data.drop(columns=['death', 'OS_time'])

In [19]:
from sksurv.ensemble import RandomSurvivalForest
from sklearn.model_selection import train_test_split
from eli5.sklearn import PermutationImportance
import eli5

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [20]:
rsf = RandomSurvivalForest()

In [21]:
def permutation(X, y, model, loops):
  rankings = np.zeros(len(X.columns),)
  bordarank = np.zeros(len(X.columns),)
  c_score_total = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)

  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same test set
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state = seed )
    X_train, X_rank, y_train, y_rank = train_test_split(
    X_train, y_train, test_size=0.5, random_state = seed)
  #Let's train the model on the 
    model.fit(X_train, y_train)
  #Let's define the permutation.
    permuter = PermutationImportance(
    estimator = model,
    n_iter = 10)
  #Let's fit the permutator on the Y test, (This permutation only shuffle the Y test.)
    permuter.fit(X_rank, y_rank)
  
    columns = X_test.columns.to_list()
  #The feature importance will give us and n array of how much the Concordance score dropped
    feature_importance = permuter.feature_importances_
  # I need to create a ranking out of this
    rankings = np.add(feature_importance, rankings)
  #This will create a numpy array with each c index loss. Each row is a loop. 
    bordarank = np.vstack((bordarank, feature_importance))
  #Let's compute the Concordance index score
    c_score = model.score(X_test, y_test)
    c_score_total.append(c_score)

  
    
  #Dividing to get the average 
  ranking_terminal = np.true_divide(rankings, loops) 
  c_rank = pd.DataFrame({'features': columns, 'C_loss': ranking_terminal}).sort_values(ascending = False, by =['C_loss'])
  c_mean = np.mean(np.array(c_score_total))
  c_std = np.std(np.array(c_score_total))
  #Let's start working with the borda. First we delete the first row that is just 0s
  bordarank = np.delete(bordarank, 0, 0)
  ranking_the_data = rankdata(bordarank * -1, axis=1)
  rankavg = np.mean(ranking_the_data, axis = 0)
  ranksd = np.std(ranking_the_data, axis = 0)
  #Vamos a ver como queda
  borda = pd.DataFrame({'Features': columns, 'C_Average_Loss': ranking_terminal, 'borda_average': rankavg, 'borda_sd': ranksd}).sort_values(ascending = True, by = ['borda_average'])
  



  return c_rank, c_mean, c_std, borda

In [22]:
rank, c_mean, c_std, borda = permutation(X, Y, rsf, 100)

In [23]:
c_mean

0.7766496154102278

In [24]:
c_std

0.01372879766960945

In [25]:
borda

,Features,C_Average_Loss,borda_average,borda_sd
3,adjuvant_treatment,0.171855,1.00,0.000000
1,biopsy_debulking,0.025196,2.42,0.635295
0,age,0.019631,2.88,0.738647
4,SVZ status,0.010251,3.93,0.724638
2,KPS,0.004276,4.77,0.420833


First top 2 features

In [26]:
X_tf = data[['adjuvant_treatment', 'biopsy_debulking']]

In [27]:
def top2features(X, y, model, loops):
  
  c_score_total = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)
  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same array of test sets, that we had in the previous feature selection algorithm. 
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state = seed )
    X_train, X_rank, y_train, y_rank = train_test_split(
    X, Y, test_size=0.5, random_state = seed)
  #Let's train the model on the 
    model.fit(X_train, y_train)
  #Let's compute the scores
    c_score = model.score(X_test, y_test)
    c_score_total.append(c_score)

  
  c_mean_top2 = np.mean(np.array(c_score_total))
  c_std_top2 = np.std(np.array(c_score_total))



  return c_mean_top2, c_std_top2

In [28]:
c_score_tf = top2features(X_tf, Y, rsf, 100)

In [29]:
c_score_tf

(0.7650182818916558, 0.011195077761728893)

## Brier Score

In [30]:
pip install pysurvival

     |████████████████████████████████| 4.7 MB 35.1 MB/s 
  Created wheel for pysurvival: filename=pysurvival-0.1.2-cp37-cp37m-linux_x86_64.whl size=3773312 sha256=463aab7e5b933b5c395566d527eb798333967c55784694eccc358228ae201cb2
  Stored in directory: /root/.cache/pip/wheels/1a/63/e2/32273d765a4e2f4ccac69c8adf97425ca80bab5d0c8447f120
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12082 sha256=18082e39fa5e823d8bdece95698cb9398e738a963b3e163fab945dae5cbd12c2
  Stored in directory: /root/.cache/pip/wheels/f0/fd/1f/3e35ed57e94cd8ced38dd46771f1f0f94f65fec548659ed855
Successfully built pysurvival progressbar


In [31]:
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.metrics import integrated_brier_score
from pysurvival.models.survival_forest import RandomSurvivalForestModel

In [32]:
T = data['OS_time']
E = data['death']

In [33]:
pyforest = RandomSurvivalForestModel(num_trees=100)

In [34]:
def Py_scores(X, T, E, model, loops):
  
  c_score_total = []
  ibstotal = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)
  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same array of test sets, that we had in the previous feature selection algorithm. 
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, t_train, t_test, e_train, e_test = train_test_split(
    X, T, E, test_size=0.33, random_state = seed )
    X_train, X_rank, t_train, t_rank, e_train, e_rank = train_test_split(
    X_train, t_train, e_train, test_size=0.5, random_state = seed)
  #Let's train the model on the 
    model.fit(X_train, t_train, e_train, max_depth=5)
    
  #Let's compute the C - scores
  #Let's compute the integrated brier score
    ibs = integrated_brier_score(model, X_test, t_test, e_test)
    ibstotal.append(ibs)
    
  
  
  ibstotal2 = np.mean(np.array(ibstotal))
  ibs_std = np.std(np.array(ibstotal))


  return ibstotal2, ibs_std

In [35]:
ibs_mean, ibs_std = Py_scores(X, T, E, pyforest, 100)

In [36]:
ibs_mean

0.1282148011699619

In [37]:
ibs_std

0.005238993029813132

## Deep Surv

In [38]:
from pysurvival.models.semi_parametric import NonLinearCoxPHModel


In [39]:
def Py_scores_deepsurv(X, T, E, loops):
  
  c_score_total = []
  ibstotal = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)
  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same array of test sets, that we had in the previous feature selection algorithm. 
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, t_train, t_test, e_train, e_test = train_test_split(
    X, T, E, test_size=0.33, random_state = seed )
    X_train, X_rank, t_train, t_rank, e_train, e_rank = train_test_split(
    X_train, t_train, e_train, test_size=0.5, random_state = seed)
  #Let's train the NN 
    structure = [ {'activation': 'ReLU', 'num_units': 64}, {'activation': 'ReLU', 'num_units': 32}, ]
    nonlinear_coxph = NonLinearCoxPHModel(structure = structure)
    nonlinear_coxph.fit(X_train, t_train, e_train, num_epochs = 500, dropout = 0.2,)
    c_score_total.append(concordance_index(nonlinear_coxph, X_test, t_test, e_test))
    
  #Let's compute the C - scores
  #Let's compute the integrated brier score
    ibs = integrated_brier_score(nonlinear_coxph, X_test, t_test, e_test)
    ibstotal.append(ibs)
    
  
  
  ibstotal2 = np.mean(np.array(ibstotal))
  ibs_std = np.std(np.array(ibstotal))
  c_score_avg = np.mean(np.array(c_score_total))
  c_score_std = np.std(np.array(c_score_total))

  return ibstotal2, ibs_std, c_score_avg, c_score_std

In [ ]:
ib, ibstd, c_index_avg, c_score_std = Py_scores_deepsurv(X, T, E, 100)

Integrated Brier Score

In [41]:
ib

0.1386942363321737

Brier score sd

In [42]:
ibstd

0.012011663805876479

In [43]:
c_index_avg

0.7311012036217651

In [44]:
c_score_std

0.036146996983189864

## Cox Proportional Hazards

In [45]:
from pysurvival.models.semi_parametric import CoxPHModel


In [46]:
coxph = CoxPHModel()

In [61]:
def Py_scores_cox(X, T, E, model, loops):
  
  c_score_total = []
  ibstotal = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)
  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same array of test sets, that we had in the previous feature selection algorithm. 
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, t_train, t_test, e_train, e_test = train_test_split(
    X, T, E, test_size=0.33, random_state = seed )
    X_train, X_rank, t_train, t_rank, e_train, e_rank = train_test_split(
    X_train, t_train, e_train, test_size=0.5, random_state = seed)
  #Let's train the model on the 
    model.fit(X_train, t_train, e_train, lr = 1e-3, max_iter = 1000)
    c_score_total.append(concordance_index(model, X_test, t_test, e_test))
    
  #Let's compute the C - scores
  #Let's compute the integrated brier score
    ibs = integrated_brier_score(model, X_test, t_test, e_test)
    ibstotal.append(ibs)
    
  
  
  ibstotal2 = np.mean(np.array(ibstotal))
  ibs_std = np.std(np.array(ibstotal))
  c_index = np.mean(np.array(c_score_total))
  c_index_sd = np.std(np.array(c_score_total))

  return ibstotal2, ibs_std, c_index, c_index_sd

In [ ]:
ib, ibstd, c_index_avg, c_index_sd = Py_scores_cox(X, T, E, coxph, 100)

In [65]:
ib

0.14789487483168215

In [66]:
ibstd

0.015178430610807606

In [64]:
c_index_avg

0.7156577669953437

In [67]:
c_index_sd

0.034909994962186594